In [186]:
#https://www.geeksforgeeks.org/removing-stop-words-nltk-python/
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
spl_characters = ['~','`','!','@','^','$','%','&','*','(',')','+','=','{','}','[',']',';',':','|','\\','"',"'",'\n','<','>',',','.','?','/','-','*','#']

f = open("imdb_labelled.txt", "r")
s_w = set(stopwords.words('english'))

strarr=[]
for line in f:
    arr=line.split("\t")
    string=arr[0]
    label=arr[1][:-1]
    for k in spl_characters:
        string=string.replace(k,' ')
    
    string=string.lower()
    obj={'str':string,'label':int(label)}
    strarr.append(obj)

for obj in strarr:
    w_t=word_tokenize(obj['str'])
    fs = [w for w in w_t if not w in s_w]
    a=" ".join(fs)
    obj['str']=a
# train test dev ratio 60 20 20
full_len=len(strarr)
len_train=int(len(strarr)*0.6)
len_dev=int(len(strarr)*0.2)
train=strarr[0:len_train]
dev=strarr[len_train:len_train+len_dev]
test=strarr[len_train+len_dev:full_len]
print(len(train)+len(test)+len(dev))


1000


In [217]:
trListAll=[]
trListPos=[]
trListNeg=[]
trSetAll=set()
trSetPos=set()
trSetNeg=set()
posDocCount=0
negDocCount=0
trSetAllCount={}
trSetPosCount={}
trSetNegCount={}


In [218]:
def train_model(data):
    global trListAll,trListPos,trListNeg,trSetAll,trSetPos,trSetNeg,posDocCount,negDocCount,trSetAllCount,trSetPosCount    
    for t in data:
        arr=word_tokenize(t['str'])
        trListAll.extend(arr)
        trSetAll.update(arr)
        if(t['label']==1):
            trListPos.extend(arr)
            trSetPos.update(arr)
            posDocCount+=1
        else:
            trListNeg.extend(arr)
            trSetNeg.update(arr)
            negDocCount+=1
            
    #count for each word in all document process        
    for twr in trSetAll:
        count=0
        for t in data:
            if twr in t['str']:
                count+=1
        trSetAllCount[twr]=count
        
    for key in trSetAllCount:
        #print(key," ",trSetAllCount[key])
        trSetAllCount[key]=trSetAllCount[key]/len(train)
        
    #count for each word in pos document process

    for twr in trSetPos:
        count=0
        for t in data:
            if(t['label']==1):
                if twr in t['str']:
                    count+=1
        trSetPosCount[twr]=count
        
     #count for each word in neg document process   
    for twr in trSetNeg:
        count=0
        for t in data:
            if(t['label']==0):
                if twr in t['str']:
                    count+=1
        trSetNegCount[twr]=count


In [219]:
def test_data(data):
    acc_count=0
    global posDocCount,negDocCount
    pos_class_prob=posDocCount/len(train)
    neg_class_prob=negDocCount/len(train)
    for d in data:
        w_t_arr=word_tokenize(d['str'])
        pos_prob=1
        neg_prob=1
        final=-1
        for w in w_t_arr:
            if(w not in trSetPosCount):
                pos_prob=pos_prob*0
            else:
                prob=trSetPosCount[w]/posDocCount
                pos_prob=pos_prob*prob
            if(w not in trSetNegCount):
                neg_prob=neg_prob*0
            else:
                prob=trSetNegCount[w]/negDocCount
                neg_prob=neg_prob*prob     
        pos_prob=pos_prob*pos_class_prob
        neg_prob=neg_prob*neg_class_prob
        if(pos_prob>neg_prob):
            final=1
        else:
            final=0
        if(final==d['label']):
            acc_count=acc_count+1
    acc_count=acc_count*100/len(data) 
    return acc_count

In [220]:
train_model(train)
train_acc=test_data(train)
test_acc=test_data(test)
print("train accuracy ",train_acc," test accuracy ",test_acc)

train accuracy  98.33333333333333  test accuracy  49.0


In [221]:
#5-fold cross validation
#https://stackoverflow.com/questions/43968369/subtracting-one-list-from-another-in-python
lendev=int(len(dev)/5)
accuracy_list=[]
for i in range(5):
    acc=0
    testdata=dev[i*lendev:(i+1)*lendev]
    #testdata=dev-traindata
    traindata= [x for x in dev if x not in testdata]
    train_model(traindata)
    acc=test_data(testdata)
    accuracy_list.append(acc)
print("5-fold accuracy list",accuracy_list)   

final_acc=0
for a in accuracy_list:
    final_acc=final_acc+a
final_acc=final_acc/len(accuracy_list)
print("5-fold accuracy ",final_acc)

5-fold accuracy list [50.0, 25.0, 65.0, 37.5, 22.5]
5-fold accuracy  40.0


In [228]:
trListAll=[]
trListPos=[]
trListNeg=[]
trSetAll=set()
trSetPos=set()
trSetNeg=set()
posDocCount=0
negDocCount=0
trSetAllCount={}
trSetPosCount={}
trSetNegCount={}
train_model(train)
acc=test_data(dev)
print("Dev accuracy without 5-fold ",acc)

Dev accuracy without 5-fold  43.5


In [223]:
def cal_acc_laplas(data,alpha):
    acc_count=0
    global posDocCount,negDocCount
    pos_class_prob=posDocCount/len(train)
    neg_class_prob=negDocCount/len(train)
    for d in data:
        w_t_arr=word_tokenize(d['str'])
        pos_prob=1
        neg_prob=1
        final=-1
        for w in w_t_arr:
            if(w not in trSetPosCount):
                pos_prob=pos_prob*alpha/(posDocCount+(2*alpha))
            else:
                prob=trSetPosCount[w]+alpha/(posDocCount+(2*alpha))
                pos_prob=pos_prob*prob
            if(w not in trSetNegCount):
                neg_prob=neg_prob*alpha/(negDocCount+(2*alpha))
            else:
                prob=trSetNegCount[w]+alpha/(negDocCount+(2*alpha))
                neg_prob=neg_prob*prob
        pos_prob=pos_prob*pos_class_prob
        neg_prob=neg_prob*neg_class_prob
        if(pos_prob>neg_prob):
            final=1
        else:
            final=0
        if(final==d['label']):
            acc_count=acc_count+1
    acc_count=acc_count*100/len(data) 
    return acc_count

In [224]:
acc1=cal_acc_laplas(dev,1)
print(acc1)
acc2=cal_acc_laplas(dev,0.1)
print(acc2)
acc3=cal_acc_laplas(dev,0.01)
print(acc3)
acc4=cal_acc_laplas(dev,0.001)
print(acc4)
acc5=cal_acc_laplas(dev,0.0001)
print(acc5)
acc6=cal_acc_laplas(dev,10)
print(acc6)
acc7=cal_acc_laplas(dev,100)
print(acc7)

69.5
69.5
69.5
69.5
69.5
69.0
71.5


In [229]:
#for 10 positive and negative words with highest probability.
p_p={}
n_p={}
for key in trSetPosCount:
    p_p[key]=trSetPosCount[key]/trSetAllCount[key]
for key in trSetNegCount:
    n_p[key]=trSetNegCount[key]/trSetAllCount[key]
# here trSetAllCount is probability of each word.divide positive probability of word with probability of word as whole
for key in p_p:
    p_p[key]=p_p[key]/trSetAllCount[key]
for key in n_p:
    n_p[key]=n_p[key]/trSetAllCount[key]



In [230]:
import operator
#https://stackoverflow.com/questions/613183/how-do-i-sort-a-dictionary-by-value 
#for sorting dictionary by value 
sorted_p_p = dict(sorted(p_p.items(), key=operator.itemgetter(1))[-10:])
print("Positive word ",sorted_p_p.keys())
sorted_n_p = dict(sorted(n_p.items(), key=operator.itemgetter(1))[-10:])
print("Negative word ",sorted_n_p.keys())


Positive word  dict_keys(['room', 'drive', 'charming', 'local', 'ladies', 'sells', 'terrific', 'impressed', 'brilliant', 'coal'])
Negative word  dict_keys(['reality', 'became', 'donlevy', 'insipid', 'reverse', 'contract', 'seemed', 'planned', 'cost', 'completed'])


In [210]:
testfinacc=cal_acc_laplas(test,100)
print(testfinacc)

75.5
